In [24]:
pip install transformers

In [25]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch
import numpy as np
import json
from multiprocessing import pool
from nltk import word_tokenize
from tqdm import tqdm
import pickle
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import LabelEncoder


In [26]:
## Load global_mapping_dict
global_mapping_dict = {}
with open('drive/MyDrive/generalized2global_mapping.json', "r") as f:
            global_mapping_dict = json.load(f)
len(global_mapping_dict)

207

In [27]:
df_test = pd.read_csv("drive/MyDrive/lab2_test_dataset.csv",sep = ';')

In [28]:

df_test = df_test.dropna()
df_test = df_test.drop_duplicates()
df_test

,index,work_name,generalized_work_class,global_work_class
0,507695.0,"монтаж шаровых кранов, дроссельной шайбы, запо...",Монтаж мк,Монтаж мк
1,464317.0,монтаж кипиа и зра с электроприводом,Монтаж ЗРА,Монтаж
2,43108.0,монтаж оборудования и приборов,Монтаж приборов,Монтаж
3,114289.0,тх./поз.2.13.1-2/монтаж арматуры,Монтаж арматуры,Монтаж
4,331380.0,"монтаж шаровых кранов, огнепреградителя, дросс...",Монтаж мк,Монтаж мк
...,...,...,...,...
197901,449736.0,электромонтажные работы,ПНР,ПНР
197902,41406.0,гидроизоляция боковая обмазочная битумная,Гидроиспытания,Гидроиспытания
197903,74913.0,"монтаж запорного клапана, кранов",Монтаж мк,Монтаж мк
197904,105922.0,монтаж оборудования,Монтаж оборудования,Монтаж


In [29]:
## Load train set and del all class with 1 row
df_train_with_pp = pd.read_csv("drive/MyDrive/data_lemmatization_spellcheck (2).csv", index_col=0)
df_train_with_pp = df_train_with_pp.dropna()
df_train_with_pp = df_train_with_pp[df_train_with_pp['generalized_work_class'] != 'Изготовление кронштейнов']
df_train_with_pp = df_train_with_pp[df_train_with_pp['generalized_work_class'] != 'Монтаж стен']
df_train_with_pp = df_train_with_pp[df_train_with_pp['generalized_work_class'] != 'Изготовление нащельников кровли']
df_train_with_pp


,work_name,generalized_work_class,global_work_class
3,монтаж лестница,Монтаж лестниц,Строительство зданий
7,контрольный измерительный прибор автоматика,Монтаж приборов,Монтаж
9,электромонтажный работа,ПНР,ПНР
10,погружение свая свая,Погружение свай,Монтаж свай
12,монтаж система водоснабжение канализация,Испытания системы водоснабжения,Испытания систем
...,...,...,...
708554,аспоза монтаж оголовок свая пожарный гидрант,Монтаж оголовков,Монтаж свай
708557,ткс монтаж запорный регулировать арматура сеть...,Монтаж ЗРА,Монтаж
708558,экс прокладка сила кабель ldb оперативный ток,Монтаж кабеля,Монтаж электрики
708588,ткс монтаж запорный регулировать арматура сеть...,Монтаж ЗРА,Монтаж


In [30]:
## Generate target for train
le = LabelEncoder()
df_train_with_pp['Target'] = le.fit_transform(df_train_with_pp['generalized_work_class'])
df_train_with_pp

,work_name,generalized_work_class,global_work_class,Target
3,монтаж лестница,Монтаж лестниц,Строительство зданий,130
7,контрольный измерительный прибор автоматика,Монтаж приборов,Монтаж,155
9,электромонтажный работа,ПНР,ПНР,185
10,погружение свая свая,Погружение свай,Монтаж свай,189
12,монтаж система водоснабжение канализация,Испытания системы водоснабжения,Испытания систем,63
...,...,...,...,...
708554,аспоза монтаж оголовок свая пожарный гидрант,Монтаж оголовков,Монтаж свай,143
708557,ткс монтаж запорный регулировать арматура сеть...,Монтаж ЗРА,Монтаж,78
708558,экс прокладка сила кабель ldb оперативный ток,Монтаж кабеля,Монтаж электрики,121
708588,ткс монтаж запорный регулировать арматура сеть...,Монтаж ЗРА,Монтаж,78


In [31]:
## Create dic for matching target-generalized_work_class
count = -1
list_target = df_train_with_pp['Target'].tolist()
dic_class_target = {}
for i  in df_train_with_pp['generalized_work_class']:
  count = count + 1
  key = list_target[count]
  dic_class_target[i] = key
len(dic_class_target)

204

In [32]:
## Load test set
df_test_with_pp = pd.read_csv("drive/MyDrive/test_lemmatization_spellcheck.csv", index_col=0)
df_test_with_pp = df_test_with_pp.dropna()
df_test_with_pp = df_test_with_pp.drop_duplicates()
df_test_with_pp

,work_name,generalized_work_class,global_work_class
0,монтаж шаров крановый дроссельный шайба запорн...,Монтаж мк,Монтаж мк
1,монтаж контрольный измерительный прибор автома...,Монтаж ЗРА,Монтаж
2,монтаж оборудование прибор,Монтаж приборов,Монтаж
3,поздний монтаж арматура,Монтаж арматуры,Монтаж
4,монтаж шаров крановый огнепреградитель дроссел...,Монтаж мк,Монтаж мк
...,...,...,...
197559,заполнение полость свая цементный песчаный сме...,Заполнение полости свай,Заполнение свай
197729,монтаж кабель кабельканал поза,Монтаж ЭОМ,Монтаж
197756,ограждениемонтаж секция заграждение,Монтаж ограждения,Монтаж
197792,кпсс погружение лидер скважина сводный план сеть,Бурение лидерных скважин (прочее),Бурение


In [33]:
## Generate test_class for comparison predict and test
test_class_list = []
for work_class in df_test_with_pp['generalized_work_class']:
  if work_class != 'Изготовление кронштейнов' and work_class != 'Монтаж стен' and work_class != 'Изготовление нащельников кровли':
    test_class = dic_class_target[work_class]
    test_class_list.append(test_class)
  else:
    test_class_list.append(1)
df_test_with_pp['test_class'] = test_class_list


In [34]:
## Split train and validation sets
X_train, X_test, y_train, y_test = train_test_split(df_train_with_pp[["work_name","global_work_class"]],df_train_with_pp["Target"], test_size=0.05, random_state=42)

In [35]:
len(np.unique(y_train))

204

In [36]:
## Convert text data to train by TfidfVectorizer
tfidfvectorizer = TfidfVectorizer()
Xt = tfidfvectorizer.fit(X_train['work_name'])
X_train_enc = Xt.transform(X_train['work_name'])
X_vaild_enc = Xt.transform(X_test['work_name'])
X_test_enc = Xt.transform(df_test_with_pp['work_name'])

Xt_df_sparse_train = pd.DataFrame.sparse.from_spmatrix(X_train_enc)
Xt_df_sparse_vaild = pd.DataFrame.sparse.from_spmatrix(X_vaild_enc)
Xt_df_sparse_test = pd.DataFrame.sparse.from_spmatrix(X_test_enc)



In [37]:
## Train

classifier = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)
classifier.fit(Xt_df_sparse_train, y_train)

yt_predict = classifier.predict(Xt_df_sparse_vaild)

In [38]:
## Preprecess validation set for evaluate metrics
X_test['Predict_class'] = yt_predict
class_list = []
global_class_predict_list = []
for predict_class in X_test['Predict_class']:
  keys = [key for key, value in dic_class_target.items() if value == predict_class]
  key = keys[0]
  class_list.append(key)
X_test["Class_name"] = class_list

for class_name in X_test["Class_name"]:
  val = global_mapping_dict[class_name]
  global_class_predict_list.append(val)
X_test["Predict_Global_name"] = global_class_predict_list
X_test.head(5)

,work_name,global_work_class,Predict_class,Class_name,Predict_Global_name
117891,бурение скважина вайс очистка полость поза,Бурение,16,Бурение скважин,Бурение
527052,некроз монтаж оборудование,Монтаж,142,Монтаж оборудования,Монтаж
215936,электроснабжение внутриплощадочный сеть плобпр...,Монтаж мк,137,Монтаж мк,Монтаж мк
201989,монтаж отстойник вода,Монтаж мк,137,Монтаж мк,Монтаж мк
37458,монтаж кабель цех,Монтаж электрики,121,Монтаж кабеля,Монтаж электрики


In [39]:
## Score for generalized_work_class in validation set
print(precision_score(yt_predict, y_test, average="macro"))
print(recall_score(yt_predict, y_test, average="macro"))

0.6866222149721848
0.7164827254406899


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
X_test_enc = Xt.transform(df_test_with_pp['work_name'])
Xt_df_sparse_test = pd.DataFrame.sparse.from_spmatrix(X_test_enc)
test_predict = classifier.predict(Xt_df_sparse_test)
test_predict

array([137,  78, 155, ..., 145,  15, 195])

In [41]:
## Preprecess test set for evaluate metrics
df_test_with_pp['Predict_class'] = test_predict
class_list = []
global_class_predict_list = []
for predict_class in df_test_with_pp['Predict_class']:
  keys = [key for key, value in dic_class_target.items() if value == predict_class]
  key = keys[0]
  class_list.append(key)
df_test_with_pp["Class_name"] = class_list

for class_name in df_test_with_pp["Class_name"]:
  val = global_mapping_dict[class_name]
  global_class_predict_list.append(val)
df_test_with_pp["Predict_Global_name"] = global_class_predict_list
df_test_with_pp.head(5)


,work_name,generalized_work_class,global_work_class,test_class,Predict_class,Class_name,Predict_Global_name
0,монтаж шаров крановый дроссельный шайба запорн...,Монтаж мк,Монтаж мк,137,137,Монтаж мк,Монтаж мк
1,монтаж контрольный измерительный прибор автома...,Монтаж ЗРА,Монтаж,78,78,Монтаж ЗРА,Монтаж
2,монтаж оборудование прибор,Монтаж приборов,Монтаж,155,155,Монтаж приборов,Монтаж
3,поздний монтаж арматура,Монтаж арматуры,Монтаж,82,82,Монтаж арматуры,Монтаж
4,монтаж шаров крановый огнепреградитель дроссел...,Монтаж мк,Монтаж мк,137,137,Монтаж мк,Монтаж мк


In [42]:
## Score for generalized_work_class in test set
print(precision_score(test_predict, df_test_with_pp['test_class'].tolist(), average="macro"))
print(recall_score(test_predict, df_test_with_pp['test_class'].tolist(), average="macro"))

0.6631826459858885
0.7216781817993458


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
## Score for global_work_class in test set
print(precision_score(df_test_with_pp["Predict_Global_name"], df_test_with_pp["global_work_class"], average="macro"))
print(recall_score(df_test_with_pp["Predict_Global_name"], df_test_with_pp["global_work_class"], average="macro"))

0.7499411099798973
0.8072639407112876


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
classifier.save_model("drive/MyDrive/Xgb_model")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:56:08] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [164]:
classifier.load_model("drive/MyDrive/Xgb_model")